## Settings

In [1]:
# 一つ上の階層からモジュールを参照できるようにする
import sys
sys.path.append('..')

In [20]:
# ロギング用
import os
from datetime import datetime, timezone, timedelta
from logs.logger import create_logger, get_logger
# モデル作成補助用
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from features.funcs import load_feather, load_feather_ohe
from evals.funcs import get_pred_result, get_acc_and_logloss, print_conf_matrix
from inputs.funcs import load_data
# モデル作成用
from models.keras import ModelKeras
from keras.layers import Dense, Dropout

In [3]:
# モジュールの変更を自動的に反映する
%load_ext autoreload
%autoreload 2

In [4]:
# loggingの設定を行う
today = datetime.now(timezone(timedelta(hours=9)))
exp_version = today.strftime('%Y%m%d')
os.environ['exp_version'] = exp_version
create_logger(exp_version)

In [5]:
os.listdir('../features/nn')

['fare_test_std_te.feather',
 'age_train_std_te.feather',
 'fare_train_std_te.feather',
 'age_test_std_mean.feather',
 'age_train_std_mean.feather',
 'age_test_std_te.feather',
 'fare_test_std_mean.feather',
 'fare_train_std_mean.feather']

## Keras

### Base

In [6]:
train_x, train_y, _ = load_data()

In [7]:
train_x = train_x.drop(['PassengerId', 'Name', 'Cabin', 'Ticket', 'Sex', 'Embarked'], axis=1)
train_x = train_x.dropna()
train_y = train_y.iloc[train_x.index]

In [8]:
model = ModelKeras(logging=True)
layer1 = Dense(1024, activation='relu', input_shape=(train_x.shape[1],))
layer2 = Dense(1024, activation='relu')
layer3 = Dense(1024, activation='relu')
layer3 = Dense(1024, activation='relu')
layer4 = Dense(1, activation='sigmoid')
model.add_layers(layer1, layer2, layer3, layer4)
model.compile()
model.set_fit_params(epochs=10, verbose=0)

[INFO] 2021-02-12 20:58:51,758 >>	=== NN KERAS MODEL ===


In [9]:
pred_train = get_pred_result(model, train_x, train_y)
get_acc_and_logloss(pred_train, train_y, logging=True)

[INFO] 2021-02-12 20:58:59,211 >>	ACCURACY: 0.7170868347338936
[INFO] 2021-02-12 20:58:59,212 >>	LOGLOSS: 0.6098663220217931
[INFO] 2021-02-12 20:58:59,213 >>	data size: 714
[INFO] 2021-02-12 20:58:59,214 >>	correct predictions: 512
[INFO] 2021-02-12 20:58:59,214 >>	


#### Age standardized and target encoding

In [129]:
train_x, train_y, test_x = load_data()

In [130]:
base_path = '../features/nn/'
cols_to_rplc_train = {
    'Age': base_path + 'age_train_std_te.feather',
    'Fare': base_path + 'fare_train_std_te.feather'
}
train_x = load_feather(train_x, cols_to_rplc_train)
train_x = load_feather_ohe(train_x, base_path + 'train_ohe.feather')
train_x = load_feather_ohe(train_x, '../features/nn/train_ohe_2_z.feather')

In [131]:
train_x = train_x.drop(['PassengerId', 'Name', 'Cabin', 'Ticket', 'Sex', 'Embarked'], axis=1)
# train_x = train_x.dropna()
# train_y = train_y.iloc[train_x.index]

In [132]:
model = ModelKeras(logging=True)
layer1 = Dense(1024, activation='relu', input_shape=(train_x.shape[1],))
layer2 = Dense(1024, activation='relu')
layer3 = Dense(1024, activation='relu')
layer3 = Dense(1024, activation='relu')
layer4 = Dense(1, activation='sigmoid')
model.add_layers(layer1, layer2, layer3, layer4)
model.compile()
model.set_fit_params(epochs=10, verbose=1)

[INFO] 2021-02-13 00:14:18,142 >>	=== NN KERAS MODEL ===


In [133]:
pred_train = get_pred_result(model, train_x, train_y)
get_acc_and_logloss(pred_train, train_y, logging=False)

Epoch 1/10
21/21 [==============================] - 0s 15ms/step - loss: 0.4628 - accuracy: 0.7814 - val_loss: 0.3941 - val_accuracy: 0.8296
Epoch 2/10
21/21 [==============================] - 0s 10ms/step - loss: 0.3601 - accuracy: 0.8548 - val_loss: 0.3805 - val_accuracy: 0.8161
Epoch 3/10
21/21 [==============================] - 0s 9ms/step - loss: 0.2567 - accuracy: 0.9057 - val_loss: 0.4325 - val_accuracy: 0.8206
Epoch 4/10
21/21 [==============================] - 0s 9ms/step - loss: 0.1570 - accuracy: 0.9281 - val_loss: 0.5186 - val_accuracy: 0.8296
Epoch 5/10
21/21 [==============================] - 0s 9ms/step - loss: 0.0538 - accuracy: 0.9790 - val_loss: 0.4682 - val_accuracy: 0.8296
Epoch 6/10
21/21 [==============================] - 0s 9ms/step - loss: 0.0369 - accuracy: 0.9865 - val_loss: 0.7208 - val_accuracy: 0.6726
Epoch 7/10
21/21 [==============================] - 0s 10ms/step - loss: 0.0370 - accuracy: 0.9835 - val_loss: 0.4414 - val_accuracy: 0.8789
Epoch 8/10
21/21 

In [134]:
model = ModelKeras(logging=True)
layer1 = Dense(1024, activation='relu', input_shape=(train_x.shape[1],))
layer2 = Dense(1024, activation='relu')
layer3 = Dense(1024, activation='relu')
layer3 = Dense(1024, activation='relu')
layer4 = Dense(1, activation='sigmoid')
model.add_layers(layer1, layer2, layer3, layer4)
model.compile()
model.set_fit_params(epochs=10, verbose=0)

[INFO] 2021-02-13 00:14:34,668 >>	=== NN KERAS MODEL ===


In [135]:
model.fit(tr_x=train_x, tr_y=train_y, va_x=train_x, va_y=train_y)

In [136]:
passenger_id = test_x['PassengerId']

In [137]:
base_path = '../features/nn/'
cols_to_rplc_train = {
    'Age': base_path + 'age_test_std_te.feather',
    'Fare': base_path + 'fare_test_std_te.feather'
}
test_x = load_feather(test_x, cols_to_rplc_train)
test_x = load_feather_ohe(test_x, base_path + 'test_ohe.feather')
test_x = load_feather_ohe(test_x, '../features/nn/test_ohe_2_z.feather')

In [138]:
test_x = test_x.drop(['PassengerId', 'Name', 'Cabin', 'Ticket', 'Sex', 'Embarked'], axis=1)

In [143]:
prediction = model.model.predict(test_x).ravel()
prediction = [int(pred) for pred in prediction]

In [144]:
submission = pd.DataFrame({'PassengerId': passenger_id, 'Survived': prediction})

In [145]:
submission_file_path = 'submissions/keras_submission_{}.csv'.format(today.strftime('%Y%m%d'))
submission.to_csv(submission_file_path, index=False)

#### Age standardized and mean

In [14]:
train_x, train_y, _ = load_data()

In [15]:
base_path = '../features/nn/'
cols_to_rplc_train = {
    'Age': base_path + 'age_train_std_mean.feather',
    'Fare': base_path + 'fare_train_std_mean.feather'
}
train_x = load_feather(train_x, cols_to_rplc_train)

In [16]:
train_x = train_x.drop(['PassengerId', 'Name', 'Cabin', 'Ticket', 'Sex', 'Embarked'], axis=1)
train_x = train_x.dropna()
train_y = train_y.iloc[train_x.index]

In [17]:
pred_train = get_pred_result(model, train_x, train_y)
get_acc_and_logloss(pred_train, train_y, logging=False)

data size: 891
correct predictions: 653
accuracy: 0.7328843996
logloss: 0.5741233327
